In [1]:
import pyodbc
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MaxAbsScaler

import warnings
warnings.filterwarnings("ignore")

from pycaret.time_series import *
from datetime import datetime

In [37]:
def df_cluster(nits_clientes, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL = pd.DataFrame()

    for nit in nits_clientes:
        #Consulta SQL
        consulta_SQL = f"SELECT DATEFROMPARTS(VTAANO, VTAMES, 1) AS 'Fecha', SUM(VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC WHERE CONCAT(CONCAT(VTANIT, '-'), VTASUC) = '{nit}' AND VTAFCH < '{fecha_final}' GROUP BY DATEFROMPARTS(VTAANO, VTAMES, 1)"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Ventas"] = df_SQL_int["Ventas"].astype(int)
        df_SQL_int["Fecha"] = pd.to_datetime(df_SQL_int["Fecha"])

        df_SQL = pd.concat([df_SQL, df_SQL_int], ignore_index=True)

    df_SQL = df_SQL.groupby("Fecha").sum()

    return df_SQL

In [38]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 12-04-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Escala R", "Escala M", "Escala F", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

filtro_distrito = 10
filtro_tipozona = "Artes graficas"
filtro_cluster = "A"

df_clusters_f = df_clusters[
                    (df_clusters["Codigo distrito"] == filtro_distrito) &
                    (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                    (df_clusters["Letra cluster"] == filtro_cluster)
                ]

In [39]:
lista_nits = df_clusters_f["Nit cliente-sucursal"].tolist()
fecha_final = '2023-03-31'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')

In [40]:
ventas_cluster = df_cluster(nits_clientes= lista_nits, fecha_final= fecha_final)

In [41]:
s = setup(ventas_cluster, fh = 6, session_id = 42)

In [42]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:51:27
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [43]:
plot_model(best, plot = 'forecast')

In [44]:
plot_model(best, plot = 'diagnostics')

In [45]:
plot_model(best, plot = 'insample')

In [46]:
plot_model(best, plot = 'forecast', data_kwargs = {'fh': 9})

### Prediction

In [47]:
# predict on test set
holdout_pred = predict_model(best)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Naive Forecaster,0.8273,0.7059,612167245.3333,687338499.4687,0.2808,0.2397,-0.2606


In [48]:
# generate forecast for 36 period in future
predict_model(best, fh = 9)

,y_pred
2022-10,2.850825e+09
2022-11,2.850825e+09
2022-12,2.850825e+09
2023-01,2.850825e+09
2023-02,2.850825e+09
2023-03,2.850825e+09
2023-04,2.850825e+09
2023-05,2.850825e+09
2023-06,2.850825e+09
